There are a lot of features that are correlated with the SalePrice.
Treat the NA as a category on it's own. If the feature is still important, it should show up.


Couple features we want to make sure.
MSSubClass -- appears to be number but it's actually categorical.

https://emredjan.github.io/blog/2017/07/11/emulating-r-plots-in-python/#leverageplot


#### 12 records within the test set are potentially problematics.
- They are index 1556, 1916, 1946, 2121, 2152, 2189, 2217, 2251, 2474, 2490, 2577, 2905 in the final combined set.
- They are Null in some fields which never existed in the train set.


In [51]:
import numpy as np
import pandas as pd

df_train = pd.read_csv('data/train.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')
df_train = df_train.set_index("Id")

df_test = pd.read_csv('data/test.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')
df_test_price = pd.read_csv('data/sample_submission.csv.gz', compression='gzip', header=0, sep=',', quotechar='"') 

test_null_columns=df_test.columns[df_test.isnull().any()] 
train_null_columns=df_train.columns[df_train.isnull().any()] 
test_null_only_ColIdx = test_null_columns.difference(train_null_columns)

test_null_only_RowIdx = [ df_test[df_test[idx].isnull()].index.tolist() for idx in test_null_only_ColIdx ]
test_null_only_RowIdx = list ( set(x for l in test_null_only_RowIdx for x in l) )

problematicTestSet = df_test.loc[ df_test.index.isin( test_null_only_RowIdx ) ]
problematicTestPrice = df_test_price.loc[ df_test_price.index.isin( test_null_only_RowIdx ) ]  #12 records
problematicTestPrice = problematicTestPrice.drop( columns=['Id'] )
problematicTestSet = pd.concat([problematicTestSet, problematicTestPrice], axis=1, sort=True)

fineTestSet = df_test.loc[ ~df_test.index.isin( test_null_only_RowIdx ) ]
fineTestPrice = df_test_price.loc[ ~df_test_price.index.isin( test_null_only_RowIdx ) ]  #1447 records
fineTestSet = pd.concat([fineTestSet, fineTestPrice], axis=1, sort=True)

problematicTestSet= problematicTestSet.set_index("Id")
fineTestSet = fineTestSet.set_index("Id")
df_test = df_test.set_index("Id")
df_test_price = df_test_price.set_index("Id")

df_test = pd.concat([df_test, df_test_price], axis=1, sort=True)
df = pd.concat([df_train,df_test], axis=0, sort=True)

#null_columns=df.columns[df.isnull().any()] 
#missCases = df[null_columns].isnull().sum() 
#print( len(missCases) -1 )  # Number of features that will have NA value, -1 is b/c NA value for the testing data.
#missCases.sort_values(ascending=False)

#df.loc[2121]

problematicTestSet.index


Int64Index([1556, 1916, 1946, 2121, 2152, 2189, 2217, 2251, 2474, 2490, 2577,
            2905],
           dtype='int64', name='Id')

#### Define a function to check out the records in the fields that we will merge and dummy them together. 
- Correct Exterior1st and Exterior2nd, Replace  :Brk Cmn  -> BrkComm and CmentBd  -> CemntBd
- Check BsmtFinType1 and BsmtFinType2. Everything is fine.

In [52]:
def checkWeirdExtra (inputDF, field1, field2):
    var2_dummy_columns = pd.get_dummies(inputDF[field1], prefix= "x")
    #print( len( var2_dummy_columns.columns ) )

    var1_dummy_columns = pd.get_dummies(inputDF[field2], prefix= "x")
    #print( len( var1_dummy_columns.columns ) )

    var_dummy_columns = pd.concat([var1_dummy_columns,var2_dummy_columns],join='outer', sort=True).groupby(level=0).sum()
    #print( len( var_dummy_columns.columns ) )

    print ( set( var_dummy_columns.columns) - set( var1_dummy_columns.columns ) )
    print ( set( var_dummy_columns.columns) - set( var2_dummy_columns.columns ) )
    
    pass


checkWeirdExtra(df, "Exterior1st", "Exterior2nd")

df.Exterior1st = df.Exterior1st.str.replace("Brk Cmn", "BrkComm")
df.Exterior2nd = df.Exterior2nd.str.replace("Brk Cmn", "BrkComm")
df.Exterior1st = df.Exterior1st.str.replace("CmentBd", "CemntBd")
df.Exterior2nd = df.Exterior2nd.str.replace("CmentBd", "CemntBd")

checkWeirdExtra(df, "BsmtFinType1", "BsmtFinType2")


{'x_WdShing', 'x_BrkComm', 'x_CemntBd'}
{'x_Brk Cmn', 'x_CmentBd', 'x_Other', 'x_Wd Shng'}
set()
set()


#### Start to purposely encode the information based on our best understanding.
- Combine Exterior1st and Exterior2nd to **Exterior**
- BsmtFinType1 and BsmtFinType2 to **Bsmt** 
    -Replace each type to it's actually square feet  BsmtFinSF1, BsmtFinSF2
    -For Unf in type 1 and type2, replace it with the BsmtUnfSF
- Combine BsmtFullBath, BsmtHalfBath to **BsmtBath**
- Add all different PorchSF to **TotalProchSF**
- Dummy MasVnrType to **MasVnr** and replace the value with MasVnrArea
- Dummy PoolQC to **Pool** and replace the value with PoolArea

In [53]:
def purposelyEncodeData(inputDF):
    
    preProcessCatField = ["PoolQC",  "MasVnrType", "Exterior1st", "Exterior2nd", "BsmtFinType1", "BsmtFinType2"]
    preProcessNumFiled = [ "PoolArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "BsmtFullBath", "BsmtHalfBath"]
    
    inputDF[preProcessCatField] = inputDF[preProcessCatField].fillna("Unknown")
    inputDF[preProcessNumFiled] = inputDF[preProcessNumFiled].fillna(-1)
    
    # Need to encode some fields based on their definition and drop the original.
    # Exterior1st, Exterior2nd (Exterior covering on house)

    var1_dummy_columns = pd.get_dummies(inputDF['Exterior1st'], prefix= "Exterior")
    var2_dummy_columns = pd.get_dummies(inputDF['Exterior2nd'], prefix= "Exterior")
    var_dummy_columns = pd.concat([var1_dummy_columns,var2_dummy_columns], join='outer', sort=True).groupby(level=0).sum()

    for k, v in var1_dummy_columns.nunique().to_dict().items():
        print('{}={}'.format(k,v))

    inputDF = pd.concat([inputDF, var_dummy_columns], join='outer', sort=True, axis=1)
    inputDF = inputDF.drop( columns=['Exterior1st', 'Exterior2nd'] )
    print( inputDF.shape )

    # BsmtFinType1, BsmtFinType2, BsmtFinSF1 (Type 1 finished square feet), BsmtFinSF2 (Type 1 finished square feet), BsmtUnfSF: Unfinished square feet of basement area
    # TotalBsmtSF: Total square feet of basement area keep
    var1_dummy_columns = pd.get_dummies(inputDF['BsmtFinType1'], prefix= "Bsmt") 
    var1_dummy_columns = var1_dummy_columns.mul( inputDF['BsmtFinSF1'] , axis=0)
    tmp = var1_dummy_columns['Bsmt_Unf']
    tmp [ inputDF.loc[inputDF['BsmtFinType1'] == "Unf"].index ] = 1
    tmp = tmp.mul( inputDF['BsmtUnfSF'] , axis=0)
    var1_dummy_columns['Bsmt_Unf'] = tmp

    var2_dummy_columns = pd.get_dummies(inputDF['BsmtFinType2'], prefix= "Bsmt") 
    var2_dummy_columns = var2_dummy_columns.mul( inputDF['BsmtFinSF2'] , axis=0)
    tmp = var2_dummy_columns['Bsmt_Unf']
    tmp [ inputDF.loc[inputDF['BsmtFinType2'] == "Unf"].index ] = 1
    tmp = tmp.mul( inputDF['BsmtUnfSF'] , axis=0)
    var2_dummy_columns['Bsmt_Unf'] = tmp

    var_dummy_columns = pd.concat([var1_dummy_columns,var2_dummy_columns], join='outer', sort=True).groupby(level=0).sum()

    for k, v in var_dummy_columns.nunique().to_dict().items():
        print('{}={}'.format(k,v))

    inputDF = pd.concat([inputDF, var_dummy_columns], join='outer', sort=True, axis=1)
    inputDF = inputDF.drop( columns=['BsmtFinType1', 'BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF'] )
    print( inputDF.shape )

    #BsmtFullBath, BsmtHalfBath  (number of type of bathroom in the basement)
    inputDF['BsmtBath'] = inputDF["BsmtFullBath"] + 0.5* inputDF["BsmtHalfBath"] 
    inputDF = inputDF.drop( columns=['BsmtFullBath', 'BsmtHalfBath'] )
    print( inputDF.shape )

    #OpenPorchSF ( Open porch area in square feet)
    #EnclosedPorch (Enclosed porch area in square feet)
    #3SsnPorch (Three season porch area in square feet)
    #ScreenPorch (Screen porch area in square feet)
    inputDF["TotalProchSF"] = inputDF["OpenPorchSF"] + inputDF["EnclosedPorch"] + inputDF["3SsnPorch"] + inputDF["ScreenPorch"] 
    print( inputDF.shape )

    #MasVnrType, MasVnrArea
    var_dummy_columns = pd.get_dummies(inputDF['MasVnrType'], prefix= "MasVnr") 
    var_dummy_columns = var_dummy_columns.mul( inputDF['MasVnrArea'] , axis=0)
    inputDF = pd.concat([inputDF, var_dummy_columns], join='outer', sort=True, axis=1)
    inputDF = inputDF.drop( columns=['MasVnrType', 'MasVnrArea'] )
    print( inputDF.shape )


    #PoolQC, PoolArea
    var_dummy_columns = pd.get_dummies(inputDF['PoolQC'], prefix= "Pool") 
    var_dummy_columns = var_dummy_columns.mul( inputDF['PoolArea'] , axis=0)
    inputDF = pd.concat([inputDF, var_dummy_columns], axis=1)
    inputDF = inputDF.drop( columns=['PoolQC', 'PoolArea'] )
    print( inputDF.shape )
    
    return (inputDF)


In [54]:
new = purposelyEncodeData(df.copy())
new.MSSubClass = new.MSSubClass.astype('object')

Exterior_AsbShng=2
Exterior_AsphShn=2
Exterior_BrkComm=2
Exterior_BrkFace=2
Exterior_CBlock=2
Exterior_CemntBd=2
Exterior_HdBoard=2
Exterior_ImStucc=2
Exterior_MetalSd=2
Exterior_Plywood=2
Exterior_Stone=2
Exterior_Stucco=2
Exterior_Unknown=2
Exterior_VinylSd=2
Exterior_Wd Sdng=2
Exterior_WdShing=2
(2919, 96)
Bsmt_ALQ=370
Bsmt_BLQ=283
Bsmt_GLQ=619
Bsmt_LwQ=199
Bsmt_Rec=302
Bsmt_Unf=1214
Bsmt_Unknown=3
(2919, 98)
(2919, 97)
(2919, 98)
(2919, 101)
(2919, 103)


In [55]:
new[ new["Bsmt_Unknown"] != 0 ].index
## index = 333, BsmtFinType2 is unknown but it has square feet for type2
## index = 2121 Nan for all the BsmtFinType series.
## The cleaning should be techinically correct.
# df [ df.index == 333 ].BsmtFinType2

Int64Index([333, 2121], dtype='int64', name='Id')

#### Checking the dummy result

In [56]:
print( new.shape )
new.dtypes
#cols_to_transform = ["MSSubClass", "MSZoning"]
#df_with_dummies = pd.get_dummies( df, columns = cols_to_transform )

(2919, 103)


1stFlrSF              int64
2ndFlrSF              int64
3SsnPorch             int64
Alley                object
BedroomAbvGr          int64
BldgType             object
BsmtCond             object
BsmtExposure         object
BsmtQual             object
CentralAir           object
Condition1           object
Condition2           object
Electrical           object
EnclosedPorch         int64
ExterCond            object
ExterQual            object
Fence                object
FireplaceQu          object
Fireplaces            int64
Foundation           object
FullBath              int64
Functional           object
GarageArea          float64
GarageCars          float64
GarageCond           object
GarageFinish         object
GarageQual           object
GarageType           object
GarageYrBlt         float64
GrLivArea             int64
                     ...   
Exterior_HdBoard      uint8
Exterior_ImStucc      uint8
Exterior_MetalSd      uint8
Exterior_Other      float64
Exterior_Plywood    

In [59]:
df = new
df_test = df[df.index >= min(df_test.index)]  ## determine if it is a test set or not. Have to encode all of them together.
df_train = df[df.index < min(df_test.index)]

Save the processed object to the pickle file

In [60]:
df_train.to_pickle("df_train.pkl")
df_test.to_pickle("df_test.pkl")
df.to_pickle("df.pkl")